In [19]:
import pandas as pd

In [20]:
df = pd.read_csv("nba_games.csv", index_col = 0)

### Clean CSV

In [21]:
df = df.sort_values("date")
df = df.reset_index(drop=True)

In [22]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

#### Add Target Column to Represent Team's Next Game

In [ ]:
def add_target(team): 
    team["target"] = team["won"].shift(-1) # create new column target that represents the team's next game result
    return team

df = df.groupby("team", group_keys=False).apply(add_target)
df["target"][pd.isnull(df["target"])] = 2 # target for last game will be 2
df["target"] = df["target"].astype(int, errors="ignore") # turn wins into 0's and 1's


#### Get Rid of Null Columns

In [26]:
nulls = pd.isnull(df).sum()
nulls = nulls[nulls > 0]
valid_cols = df.columns[~df.columns.isin(nulls.index)]
df = df[valid_cols].copy()
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,0.769,...,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True,1
1,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,...,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False,1
2,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,...,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False,1
3,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,...,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True,1
4,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,...,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21819,240.0,47.0,94.0,0.500,16.0,39.0,0.410,10.0,13.0,0.769,...,37.2,200.0,129.0,PHO,136,0,2024,2024-01-31,False,2
21820,240.0,49.0,79.0,0.620,14.0,28.0,0.500,24.0,32.0,0.750,...,28.6,214.0,146.0,BRK,120,1,2024,2024-01-31,True,2
21821,240.0,47.0,100.0,0.470,12.0,30.0,0.400,19.0,22.0,0.864,...,35.5,207.0,136.0,WAS,109,1,2024,2024-01-31,True,2
21822,240.0,41.0,93.0,0.441,12.0,40.0,0.300,12.0,18.0,0.667,...,27.6,175.0,116.0,MIA,115,1,2024,2024-01-31,False,2


### ML Pipeline

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split) # start with 0 features, pick best and continue to 30


In [ ]:
remove_cols = ["season", "date", "won", "target", "team", "team_opp"]
select_cols = df.columns[~df.columns.isin(remove_cols)]